In [4]:
import cv2
import torch
import time

# Load the YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Path to the video file
video_path = r'C:\Users\kligh\OneDrive\Desktop\Machine_Learning_Projects()\Traffic_Control_YoloV5\trafficnet_dataset_v1\traffic_video_test_1.mp4'

# Function to simulate traffic light control for one direction
def traffic_light_control_one_direction(video_path, min_green_duration=10, max_green_duration=30, vehicle_count_threshold=5):
    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Check if the video opened successfully
    if not cap.isOpened():
        print(f"Error: Could not open video file {video_path}")
        return

    # Initialize light state and timer for the single direction (NS)
    green_light_on = True  # Start with green light for the direction
    light_timer = 0
    last_switch_time = 0

    # Define minimum and maximum durations for green lights
    green_duration_dynamic = min_green_duration

    # Get video frame width and height
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))

    # Optionally, define the codec and create VideoWriter object to save the output video
    output_path = r'C:\Users\kligh\OneDrive\Desktop\FCS - FP\Traffic Detection\result\traffic_light_control_output_one_direction.mp4'
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), 30, (frame_width, frame_height))

    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()

        if not ret:
            print("End of video or failed to grab frame.")
            break

        # Perform YOLOv5 inference on the frame
        results = model(frame)

        # Get the detected objects' bounding boxes and labels
        frame_with_detections = results.render()[0]

        # Make a writable copy of the rendered frame
        frame_with_detections = frame_with_detections.copy()

        # Extract the detected object class IDs
        detected_classes = results.xyxy[0][:, -1].cpu().numpy()

        # Define vehicle-related class names
        vehicle_classes = ['car', 'bus', 'truck']

        # Count vehicles in the current frame (one direction)
        vehicle_count = 0

        for i, cls in enumerate(detected_classes):
            class_name = results.names[int(cls)]
            bbox = results.xyxy[0][i][:4]  # Bounding box [x1, y1, x2, y2]
            if class_name in vehicle_classes:
                vehicle_count += 1

        # Calculate traffic density (you can improve this later)
        traffic_density = vehicle_count / frame_width

        # Dynamic green light duration adjustment based on vehicle count
        current_time = time.time()
        elapsed_time = current_time - last_switch_time

        # Adjust green light duration based on the vehicle count
        green_duration_dynamic = min(max(min_green_duration, vehicle_count * 2), max_green_duration)

        # Light logic
        if green_light_on:
            if elapsed_time >= green_duration_dynamic:
                # Switch to red light
                green_light_on = False
                last_switch_time = current_time
        else:
            # The light is red, maintain red for a set time
            if elapsed_time >= min_green_duration:
                # Switch back to green light
                green_light_on = True
                last_switch_time = current_time

        # Add vehicle count and light status text to the frame
        cv2.putText(frame_with_detections, f'Vehicle Count: {vehicle_count}', (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(frame_with_detections, f'Green Light: {"ON" if green_light_on else "OFF"}', (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0) if green_light_on else (0, 0, 255), 2, cv2.LINE_AA)

        # Display the frame with detections and traffic light control
        cv2.imshow('Traffic Light Control (One Direction)', frame_with_detections)

        # Save the frame to the output video
        out.write(frame_with_detections)

        # Press 'q' to quit the video viewer
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the video file and writer objects
    cap.release()
    out.release()

    # Close all OpenCV windows
    cv2.destroyAllWindows()

# Process the video file for one-direction traffic light control simulation
traffic_light_control_one_direction(video_path)


Using cache found in C:\Users\kligh/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-10-19 Python-3.9.18 torch-2.4.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
C:\Users\kligh/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\kligh/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\kligh/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\kligh/.cache\torch\hub\ultralytics_yolov5_m

End of video or failed to grab frame.


C:\Users\kligh/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
